In [1]:
!pip install gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.2/57.2 MB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.4/320.4 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/94.8 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.3/11.3 MB 33.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.2/73.2 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 2.3 MB/s eta 0:00:00
  Attempting uninstall: markupsafe
    Found existing installation: MarkupSafe 3.0.2
    Uninstalling MarkupSafe-3.0.2:
      Successfully uninstalled MarkupSafe-3.0.2


In [2]:
import torch
from sentence_transformers import SentenceTransformer, util
import gradio as gr

In [3]:
# Step 1: Load a pretrained model for semantic search
model = SentenceTransformer('all-MiniLM-L6-v2')  # A lightweight LLM from Hugging Face for embeddings

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [4]:
# Example dataset of courses with titles, descriptions, and links
courses = [
    {"title": "Framework to Choose the Right LLM for your Business", "description": "This course provides a comprehensive framework for selecting the right LLM for your business. Learn to evaluate LLMs based on accuracy, cost, scalability, and more, while exploring real-world applications to make informed, strategic AI decisions.", "link": "https://courses.analyticsvidhya.com/courses/choosing-the-right-LLM-for-your-business"},
    {"title": "Improving Real World RAG Systems: Key Challenges & Practical Solutions", "description": "Master key challenges in real-world Retrieval-Augmented Generation (RAG) systems. Explore practical solutions, advanced retrieval strategies, and agentic RAG systems to improve context, relevance, and accuracy in AI-driven applications.", "link": "https://courses.analyticsvidhya.com/courses/improving-real-world-rag-systems-key-challenges"},
    {"title": "Building Smarter LLMs with Mamba and State Space Model", "description": "Master Mamba's selective state space model for LLMs. Discover key components like the Mamba block, optimizing sequence modeling with efficient, scalable training and inference, surpassing traditional Transformers.", "link": "https://courses.analyticsvidhya.com/courses/building-smarter-llms-with-mamba-and-state-space-model"},
    {"title": "Generative AI - A Way of Life - Free Course", "description": "Embark on a journey into Generative AI for beginners. Learn AI-powered text and image generation, use top AI tools, and explore industry applications. Gain practical skills, understand ethical practices, and master prompting techniques", "link": "https://courses.analyticsvidhya.com/courses/genai-a-way-of-life"},
    {"title": "Getting Started with Large Language Models", "description": "Embarking on a journey into Large Language Models (LLMs) can be seamless with the right approach. This course offers an optimal pathway to delve into the intricacies of natural language processing and model training.", "link": "https://courses.analyticsvidhya.com/courses/getting-started-with-llms"},
    {"title": "Building LLM Applications using Prompt Engineering - Free Course", "description": "This free course offers a comprehensive guide on building LLM applications, mastering prompt engineering, implementing best practices, and developing chatbots using enterprise data using advanced techniques like few-shot or one-shot prompting", "link": "https://courses.analyticsvidhya.com/courses/building-llm-applications-using-prompt-engineering-free"},
    {"title": "MidJourney: From Inspiration to Implementation - Free Course", "description": "nderstand the fundamentals of the famous image generation tool - Midjourney in this free course. You will learn the various components of Midjourney and how to use it to bring your imaginations to real world.", "link": "https://courses.analyticsvidhya.com/courses/midjourney_from_inspiration_to_implementation"},
    {"title": "Building Your first RAG System using LlamaIndex - Free Course", "description": "Dive into the sea of Retrieval Augmented Generation with this free course that will help you build your first RAG model with the popular LlamaIndex.", "link": "https://courses.analyticsvidhya.com/courses/building-first-rag-systems-using-llamaindex"},
    {"title": "Exploring Stability.AI - Free Course", "description": "Explore Stability.AI with this free course providing hands-on experience. Learn to deploy SD WebUI, use Automatic WebUI on RunPod GPU environments, and master installation, setup, generation, and customization of SD.", "link": "https://courses.analyticsvidhya.com/courses/exploring-stability-ai"},
    {"title": "Introduction to AI & ML", "description": "Artificial Intelligence (AI) and Machine Learning (ML) are changing the world around us. From functions to industries, AI and ML are disrupting how we work and how we function. Get to know all about the different facets of AI and ML in this course.", "link": "https://courses.analyticsvidhya.com/courses/introduction-to-ai-ml"},
    {"title": "Introduction to Python", "description": "Power up your career with the best and most popular data science language, Python. Leverage your Python skills to start your Data Science journey. This course is intended for beginners with no coding or Data Science background.", "link": "https://courses.analyticsvidhya.com/courses/introduction-to-data-science"},
    {"title": "Machine Learning Certification Course for Beginners", "description": "In this free machine learning certification course, you will learn Python, the basics of machine learning, how to build machine learning models, and feature engineering techniques to improve the performance of your machine learning models.", "link": "https://courses.analyticsvidhya.com/courses/Machine-Learning-Certification-Course-for-Beginners"},
    {"title": "The Working of Neural Networks - Free Course", "description": "Kickstarting your career in the field of Deep Learning can be made easy with the right guide. This course will serve as a learning path to help beginners navigate through the complex terrain of Deep Learning.", "link": "https://courses.analyticsvidhya.com/courses/The%20Working%20of%20Neural%20Networks"},
    {"title": "Understanding Linear Regression - Free Course", "description": "Linear Regression is one of the foundational algorithms used in the industry to make predictions for regression problems. In this free course let us understand Linear Regression using a Business Case Study", "link": "https://courses.analyticsvidhya.com/courses/free-understanding-linear-regression"},
    {"title": "Building a Text Classification Model with Natural Language Processing - Free Course", "description": "Explore the essentials of NLP with our comprehensive course! Learn text classification, NLP models with PyTorch, and real-world applications. Perfect for beginners and professionals alike, dive into AI-driven text analysis and hands-on projects.", "link": "https://courses.analyticsvidhya.com/courses/free-building-textclassification-natural-language-processing"},
    {"title": "The A to Z of Unsupervised ML - Free Course", "description": "Get ahead of the crowd with this free course on Unsupervised Machine Learning Models. We will be covering popular clustering algorithms and DBSCAN and show you its applications on a real-world business problem.", "link": "https://courses.analyticsvidhya.com/courses/free-unsupervised-ml-guide"},
    {"title": "Data Preprocessing on a Real-World Problem Statement - Free Course", "description": "Kickstarting your career in Data Science can be easy with the right guide. This free course will serve as the perfect path to help beginners navigate the complex terrain of Data Preprocessing and prepare any data for modelling.", "link": "https://courses.analyticsvidhya.com/courses/data-preprocessing"},
    {"title": "Introduction to Business Analytics", "description": "Business analytics is thriving and so is its role in forward-thinking organizations around the world. The demand for business analytics professionals is growing multifold - and now is the time to start working towards your desired career.", "link": "https://courses.analyticsvidhya.com/courses/introduction-to-analytics"},
    {"title": "Microsoft Excel: Formulas & Functions", "description": "Microsoft Excel is still the tool of choice in the industry when it comes to performing data analysis, thanks to its incredible depth and array of formulas and functions. This course covers a wide range of Excel formulas, including LookUp Functions!", "link": "https://courses.analyticsvidhya.com/courses/microsoft-excel-formulas-functions"},
    {"title": "Tableau for Beginners", "description": "Tableau is the tool of choice for business intelligence, analytics and data visualization experts. Learn how to use Tableau, the different features of Tableau, and start building impactful visualization using this Tableau tutorial!", "link": "https://courses.analyticsvidhya.com/courses/tableau-for-beginners"},
    {"title": "Loan Prediction Practice Problem (Using Python)", "description": "This course is aimed for people getting started into Data Science and Machine Learning while working on a real life practical problem.", "link": "https://courses.analyticsvidhya.com/courses/loan-prediction-practice-problem-using-python"},
    {"title": "Twitter Sentiment Analysis", "description": "What is sentiment analysis? Why is sentiment analysis so popular in data science? And how can you perform sentiment analysis? Find the answers to all these questions in this free course on Sentiment Analysis using Python!", "link": "https://courses.analyticsvidhya.com/courses/twitter-sentiment-analysis"},
    {"title": "Introduction to Web Scraping using Python", "description": "What is web scraping? Why is web scraping a must-know skill? How can you perform web scraping in Python? This course will cover all these aspects of web scraping and showcase how to perform web scraping using BeautifulSoup and Scrapy.", "link": "https://courses.analyticsvidhya.com/courses/introduction-to-web-scraping"},
    {"title": "Big Mart Sales Prediction Using R", "description": "This course is aimed for people getting started into Data Science and Machine Learning while solving the Big Mart Sales Prediction problem.", "link": "https://courses.analyticsvidhya.com/courses/big-mart-sales-prediction-using-r"},
    {"title": "Time Series Forecasting using Python", "description": "Learn time series analysis and build your first time series forecasting model using ARIMA, Holt’s Winter and other time series forecasting methods in Python for a real-life industry use case", "link": "https://courses.analyticsvidhya.com/courses/creating-time-series-forecast-using-python"}
    # Here we can add more course entries with titles, descriptions, and links here...
]

In [5]:
# Combine course titles and descriptions
course_texts = [f"{course['title']} - {course['description']}" for course in courses]

In [6]:
# Step 2: Precompute embeddings for all courses
course_embeddings = model.encode(course_texts, convert_to_tensor=True)


In [7]:
# Step 3: Define a search function that includes clickable course links in the output
def smart_search(query):
    # Convert query to embedding
    query_embedding = model.encode(query, convert_to_tensor=True)

    # Compute cosine similarities between query and course embeddings
    similarities = util.pytorch_cos_sim(query_embedding, course_embeddings)[0]

    # Find the top 3 most relevant courses
    top_results = torch.topk(similarities, k=5)  # Adjust 'k' as needed

    # Format results for display using Markdown to make the links clickable and titles bold
    results = []
    for idx in top_results.indices:
        course = courses[idx]
        # Format as a Markdown link with the title in bold
        result_text = f"**[{course['title']}]({course['link']})**: {course['description']}"
        results.append(result_text)

    # Join results with a newline for display
    return "\n\n".join(results)

In [8]:
# Step 4: Create a Gradio interface with Markdown output enabled
interface = gr.Interface(
    fn=smart_search,
    #inputs="text",
    inputs=gr.Textbox(lines=1, placeholder="Enter your search query......"),
    outputs="markdown",  # Use markdown output to enable clickable links and formatting
    title="Smart Course Search Tool",
    description="Type a query to find the top 5 most relevant free courses on Analytics Vidhya."
)


In [9]:
# Step 5: Launch the app on Hugging Face Spaces
if __name__ == "__main__":
    interface.launch(share=True)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://db93818a4a7b8d6b6f.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
